<a href="https://colab.research.google.com/github/GithubofRuZhang/Robust-Princing-Optimization/blob/main/Large_Scale_Price_Optimization_via_Network_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import networkx as nx

def construct_graph_and_compute_min_cut(A, b, C):
    """
    使用networkx构建加权有向图并计算最小s-t割。
    """
    n = A.shape[0]
    G = nx.DiGraph()

    G.add_node('s')
    G.add_node('t')
    for i in range(1, n+1):
        G.add_node(i)

    for i in range(n):
        for j in range(n):
            if A[i, j] < 0:  # 确保添加的边权重为正值
                G.add_edge(i+1, j+1, capacity=-A[i, j])
        G.add_edge('s', i+1, capacity=max(b[i], 0))
        G.add_edge(i+1, 't', capacity=max(-b[i], 0))

    for u, v in C:
        G.add_edge(u, v, capacity=float('inf'))

    cut_value, (reachable, non_reachable) = nx.minimum_cut(G, 's', 't')

    x_star = np.zeros(n)
    for i in range(1, n+1):
        if i in reachable:
            x_star[i-1] = 1

    return x_star

def iterative_relaxation_algorithm(A, b, C):
    n = A.shape[0]
    Gamma_t = np.ones((n, n)) / 2
    min_value = np.inf
    psi = -np.inf

    for t in range(1, 100):  # 迭代次数上限，实际应用中可根据需要调整
        x_t = construct_graph_and_compute_min_cut(A, b, C)

        value_t = x_t.T @ A @ x_t + b.T @ x_t
        A_plus = np.linalg.pinv(A)
        h_Gamma_t = x_t.T @ (A_plus * Gamma_t) @ np.ones(n) + np.ones(n).T @ (A_plus * Gamma_t) @ x_t - np.ones(n).T @ (A_plus * Gamma_t) @ np.ones(n)
        psi_t = x_t.T @ A_plus @ x_t + b.T @ x_t + h_Gamma_t
        psi = max(psi, psi_t)

        if value_t < min_value:
            min_value = value_t
            x_hat = x_t

        S_t = A_plus * (np.outer(np.ones(n), np.ones(n)) - np.outer(x_t, np.ones(n)) - np.outer(np.ones(n), x_t))
        eta_t = 0.01  # 学习率，需要根据具体问题进行调整
        Gamma_t_next = np.clip(Gamma_t - eta_t * S_t, 0, 1)

        if np.linalg.norm(Gamma_t_next - Gamma_t, 'fro') < 1e-4:  # 收敛判断
            break

        Gamma_t = Gamma_t_next

    return x_hat, min_value, psi




### 问题实例

假设我们面对的是一个二次规划问题，目标是最小化下列目标函数，同时满足二进制决策变量和特定的序列约束：

- **目标函数**:
    $$
    \text{Minimize } \mathbf{x}^{\top} A \mathbf{x} + \mathbf{b}^{\top} \mathbf{x}
    $$
    其中，$A$ 是一个 $3 \times 3$ 矩阵，$b$ 是一个长度为 3 的向量。

- **约束**:
    - $x_i \in \{0, 1\}$ 对于所有 $i$。
    - $x_1 \leq x_2$, $x_2 \leq x_3$，即 $C$ 包含边缘 $(1, 2)$ 和 $(2, 3)$。

### 实现

首先，我们定义 $A$, $b$, 和 $C$：

```python
import numpy as np

# 定义矩阵A和向量b
A = np.array([[-1, 0, 0], [0, -2, 0], [0, 0, -3]])
b = np.array([1, 2, 3])

# 定义约束集合C
C = [(1, 2), (2, 3)]


In [ ]:
# 示例问题实例
import numpy as np

# 定义矩阵A和向量b
A = np.array([[-1, 0, 0], [0, -2, 0], [0, 0, -3]])
b = np.array([1, 2, 3])

# 定义约束集合C
C = [(1, 2), (2, 3)]


x_hat, min_value, psi = iterative_relaxation_algorithm(A, b, C)

print("Approximate solution x_hat:", x_hat)
print("Minimum value:", min_value)
print("Lower bound psi:", psi)

Approximate solution x_hat: [1. 1. 1.]
Minimum value: 0.0
Lower bound psi: 4.10388888888889
